<h1> Etape 3 : Data Cleaning </h1>

Etape très importante qui vise à analyser chaque colonne du dataset et de les modifier si besoin pour : 
- Ajuster la colonne à prédire <br>
- Corriger le format des colonnes <br>
- Supprimer les colonnes avec trop de valeurs manquantes <br>
- Corriger des valeurs manquantes ou erronées <br>
- Faire du feature modeling

In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
data = pd.concat([train,test],sort=True)
data.reset_index(inplace=True)
data.head(5)

<h1> Préparation de la donnée </h1>

<b> La valeur à prédire : le prix de la maison </b>

In [ ]:
data.columns

In [ ]:
print(data['SalePrice'].isnull().sum())
print(train['SalePrice'].isnull().sum())

In [ ]:
train['SalePrice'].describe()

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns

sns.distplot(train['SalePrice'])
plt.title('Kurtosis : ' +str(train['SalePrice'].kurtosis())+ ', Skewness :' + str(train['SalePrice'].skew()))
plt.show()
print(train['SalePrice'].kurtosis())

<b> Les autres colonnes </b> <br><br>
Ont-elles le bon format ?

In [ ]:
data.drop(['SalePrice'],axis=1, inplace=True)

In [ ]:
data['1stFlrSF'].head(5)

In [ ]:
list(data['1stFlrSF'].head(5))

In [ ]:
echantillonColonnes = []
for i in data.columns:
    listcolumn = str(list(data[i].head(5)))
    echantillonColonnes.append(listcolumn)
echantillonColonnes[0:5]

In [ ]:
list(data.dtypes[0:5])

In [ ]:
d = {'colonne': list(data.columns), 'type': list(data.dtypes), 'Echantillon':echantillonColonnes}
colonnesTypes = pd.DataFrame(data=d)

pd.options.display.max_rows = 81
colonnesTypes

<b> Les autres colonnes comportent-elles des valeurs manquantes </b> <br> <br>
La régression linéaire n'accèpte pas les valeurs manquantes, nous devons donc néttoyé le Data set 

In [ ]:
data_na = (data.isnull().sum() / len(data)) * 100
data_na

In [ ]:
data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' : data_na})
missing_data

Pas de règle universelle mais si une colonne à plus de 50% de données manquantes il faut la retirer du dataset. Mais si on regarde la définition de PoolQC, MiscFeature, Alley, Fence et FireplaceQu 'Na' veut dire 'Pas de piscine', 'Pas de caractéristique particulière', 'Pas d'allée etc'...c'est donc bien une information ! 

In [ ]:
data[['PoolQC','MiscFeature','Alley','Fence','FireplaceQu']].head(8)

In [ ]:
data[['PoolQC','MiscFeature','Alley','Fence','FireplaceQu']] = data[['PoolQC','MiscFeature','Alley','Fence','FireplaceQu']].fillna("None")
data[['PoolQC','MiscFeature','Alley','Fence','FireplaceQu']].head(8)

LotFrontage = ligne de facade, c'est à dire la longueur du terrain commun à la rue.<br>
Première méthode : remplacer une variable numérique manquante par sa médiane (adaptée au data set avec des valeurs abérentes) ou sa moyenne (adapté aux data set sans valeur abérente).

In [ ]:
sns.distplot(data['LotFrontage'][data['LotFrontage'].notnull()])

In [ ]:
sns.distplot(data['LotFrontage'][data['LotFrontage']>150])

In [ ]:
data['LotFrontage'].median()

In [ ]:
data['LotFrontage'] = data['LotFrontage'].fillna(data['LotFrontage'].median())
data['LotFrontage'].head(10)

Toutes les valeurs manquantes relatives aux garages indiquent en réalité l'absence de garage

In [ ]:
data[['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond','GarageYrBlt', 'GarageArea', 'GarageCars']].head(10)

Pour les valeurs qualitatives nous allons placer 'None' pour toute information manquante. <br>
Pour les valeurs quantitatives nous allons placer '0' pour toute information manquante

In [ ]:
data[['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']] = data[['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']].fillna("None")
data[['GarageYrBlt', 'GarageArea', 'GarageCars']] = data[['GarageYrBlt', 'GarageArea', 'GarageCars']].fillna(0)

On continue ce travail sur toutes les variables suivantes

In [ ]:
data[['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath']] = data[['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath']].fillna(0)
data[['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']] = data[['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']].fillna("None")
data["MasVnrType"] = data["MasVnrType"].fillna("None")
data["MasVnrArea"] = data["MasVnrArea"].fillna(0)
data['MSZoning'] = data['MSZoning'].fillna(data['MSZoning'].mode()[0])
data = data.drop(['Utilities'], axis=1)
data["Functional"] = data["Functional"].fillna("Typ")
data['Electrical'] = data['Electrical'].fillna(data['Electrical'].mode()[0])
data['KitchenQual'] = data['KitchenQual'].fillna(data['KitchenQual'].mode()[0])
data['Exterior1st'] = data['Exterior1st'].fillna(data['Exterior1st'].mode()[0])
data['Exterior2nd'] = data['Exterior2nd'].fillna(data['Exterior2nd'].mode()[0])
data['SaleType'] = data['SaleType'].fillna(data['SaleType'].mode()[0])
data['MSSubClass'] = data['MSSubClass'].fillna("None")

Et on vérifie finalement si certaines valeurs sont encore manquantes

In [ ]:
data_na = (data.isnull().sum() / len(data)) * 100
data_na

Aller plus loin :https://towardsdatascience.com/handling-missing-values-in-machine-learning-part-1-dda69d4f88ca

<b> Est-ce que l'information contenue dans une des colonnes ne peut pas être correlée au prix de la maison ? </b>

In [ ]:
data['Id'].head(5)

In [ ]:
data.drop(['Id'],axis=1, inplace=True)

<b>Est-ce que certaines informations numériques sont réellement numériques ? <b>

In [ ]:
data[['MSSubClass','OverallCond','YrSold','MoSold']].head(5)

- MSSubClass: The building class <br>
- OverallCond: Overall condition rating <br>
- MoSold: Month Sold <br>
- YrSold: Year Sold <br>

In [ ]:
data[['MSSubClass','OverallCond','YrSold','MoSold']] = data[['MSSubClass','OverallCond','YrSold','MoSold']].astype('object')
data[['MSSubClass','OverallCond','YrSold','MoSold']].head(5)

Est-ce qu'il n'est pas possible ici de faire du feature modeling ?

In [ ]:
data[['YearBuilt','YearRemodAdd']].head(5)

In [ ]:
import datetime
current_date = datetime.datetime.now().year
current_date

In [ ]:
data['years since last remod'] = current_date - data['YearRemodAdd']
data.drop(['YearBuilt','YearRemodAdd'], axis=1, inplace = True)
data['years since last remod'].head(3)

<b>Préparer ses datasets de test et d'entrainement<b>

In [ ]:
train.head(3)

In [ ]:
test.head(3)

In [ ]:
dataset = data.loc[:1459,:]

In [ ]:
pd.options.mode.chained_assignment = None
dataset['SalePrice'] = train['SalePrice']

In [ ]:
dataset.head(3)

In [ ]:
dataset.drop(['index'],axis=1, inplace=True)

In [ ]:
import random
random_indices = random.sample(range(0, len(dataset)),k=len(dataset))
dataset.iloc[random_indices].head(5)

In [ ]:
random.seed(1)
random_indices = random.sample(range(0, len(dataset)),k=len(dataset))
datasetR = dataset.iloc[random_indices]
datasetR.head(5)

In [ ]:
datasetR.reset_index(inplace=True)
datasetR.drop(['index'],axis=1, inplace=True)
datasetR.head(5)

In [ ]:
cut = round(len(datasetR)*0.8)
cut

In [ ]:
dsTrain = datasetR.iloc[0:cut,:]
dsTest = datasetR.iloc[cut:,:]

<b>Créer son premier modèle de prédiction avec une regression univariée</b>

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = datasetR['GrLivArea'], y = datasetR['SalePrice'])
plt.ylabel('SalePrice')
plt.xlabel('GrLivArea')
plt.show()

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

lr = linear_model.LinearRegression() #quel type de modèle je souhaite utiliser ?

lr.fit(dsTrain[['GrLivArea']],dsTrain['SalePrice']) # entrainement du modèle

prediction = lr.predict(dsTest[['GrLivArea']]) # réaliser la prédiction
prediction[0:5]

In [ ]:
rmse = mean_squared_error(prediction,dsTest['SalePrice'])**(1/2) # calculer l'erreur
rmse

In [ ]:
def linearRegression(datasetR,fit,toPredict):
    
    cut = round(len(datasetR)*0.8)
    dsTrain = datasetR.iloc[0:cut,:].reset_index()
    dsTest = datasetR.iloc[cut:,:]
    
    lr = linear_model.LinearRegression() 
    lr.fit(dsTrain[fit],dsTrain[toPredict])
    prediction = lr.predict(dsTest[fit])
    rmse = mean_squared_error(prediction,dsTest[toPredict])**(1/2)
    return rmse

linearRegression(datasetR,['GrLivArea'],'SalePrice')

<b> Standardisation des valeurs </b>

In [ ]:
datasetR.select_dtypes(include=['int64','float64']).head(3)

In [ ]:
fullListNumbersAndPrice = list(datasetR.select_dtypes(include=['int64','float64']).columns)
#print(fullListNumbersAndPrice)

fullListNumbers = fullListNumbersAndPrice[:-1]
print(fullListNumbers)

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
dfStand = scaler.fit_transform(datasetR[fullListNumbers])
dfStand = pd.DataFrame(dfStand, columns=fullListNumbers)
dfStand.head(5)

In [ ]:
def linearRegression(datasetR,fit,toPredict):
    fullListNumbers = list(datasetR.select_dtypes(include=['int64','float64']).columns)
    fullListNumbers = fullListNumbers[:-1]
    
    scaler = preprocessing.StandardScaler()
    dfStand = scaler.fit_transform(datasetR[fullListNumbers])
    dfStand = pd.DataFrame(dfStand, columns=fullListNumbers)
    datasetR[fullListNumbers] = dfStand
    
    cut = round(len(datasetR)*0.8)
    dsTrain = datasetR.iloc[0:cut,:].reset_index()
    dsTest = datasetR.iloc[cut:,:]
    
    lr = linear_model.LinearRegression() 
    lr.fit(dsTrain[fit],dsTrain[toPredict])
    prediction = lr.predict(dsTest[fit])
    rmse = mean_squared_error(prediction,dsTest[toPredict])**(1/2)
    return rmse

linearRegression(datasetR,['GrLivArea'],'SalePrice')

<b> Regression linéaire avec plusieurs colonnes d'entrainement </b>

In [ ]:
datasetR['GarageCars'].head(10)

In [ ]:
fit = ['GrLivArea','GarageCars']
linearRegression(datasetR,fit,'SalePrice')

Youhou ! on a plus qu'à ajouter toutes les colonnes ! 

In [ ]:
results = []
for i in range(len(fullListNumbers)):
    fit = fullListNumbers[0:i+1]
    result = linearRegression(datasetR,fit,'SalePrice')
    results.append(result)

In [ ]:
import numpy as np
print(min(results))
plt.figure(figsize=(12, 8))
sns.lineplot(y=results, x=list(np.arange(0,len(fullListNumbers))))
plt.show()

<b> Faire le tri parmi les features numériques </b> <br>
<p> Ecarter les colonnes avec une corrélation faible avec le prix <p>

In [ ]:
datasetR[fullListNumbersAndPrice].corr().head(5)

In [ ]:
df_corr = datasetR[fullListNumbersAndPrice].corr()
corrSalesprice = df_corr.sort_values('SalePrice',ascending=False)
plt.figure(figsize=(12, 8))
sns.barplot(x=corrSalesprice['SalePrice'], y=corrSalesprice.index)
plt.show()

In [ ]:
df_corr = abs(datasetR[fullListNumbersAndPrice].corr())
corrSalesprice = df_corr.sort_values('SalePrice',ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x=corrSalesprice['SalePrice'], y=corrSalesprice.index)
plt.show()

0.00-0.19: very weak <br>
0.20-0.39: weak <br>
0.40-0.59: moderate <br>
0.60-0.79: strong <br>
0.80-1.00: very strong.

In [ ]:
less20Percent = list(corrSalesprice[corrSalesprice['SalePrice'] > 0.2].index)
less20Percent.remove('SalePrice')
print(less20Percent)

less40Percent = list(corrSalesprice[corrSalesprice['SalePrice'] > 0.4].index)
less40Percent.remove('SalePrice')
print(less40Percent)

In [ ]:
print(linearRegression(datasetR,fullListNumbers,'SalePrice'))
print(linearRegression(datasetR,less20Percent,'SalePrice')) #Winner ! 
print(linearRegression(datasetR,less40Percent,'SalePrice'))

Supprimer les colonnes trop similaires

In [ ]:
less20Percent.append('SalePrice')
fullListNumbers = less20Percent
df_corr = abs(datasetR[fullListNumbers].corr())   
plt.figure(figsize=(10, 7))
sns.heatmap(df_corr)
plt.show()

On voit que certaines caractéristiques semblent liés : <br>

- Total bsmt SF (Total square feet of basement area) et 1st Flr SF (First Floor square feet)
- Gr Liv Area (Above grade (ground) living area square feet) et TotRms AbvGrd (Total rooms above grade (does not include bathrooms)
- Garage cars (Size of garage in car capacity) et garage Area (Garage Area)

Afin d'éviter d'avoir des informations qui se doublonne et donc ne faire qu'ajouter du bruit l'une à l'autre, nous n'allons conservé que celles ayant le plus fort taux de correlation de chaque couple avec le prix :

- Total bsmt SF
- Gr Liv Area
- GarageArea

In [ ]:
fullListNumbers.remove('1stFlrSF')
fullListNumbers.remove('TotRmsAbvGrd')
fullListNumbers.remove('GarageCars')
#'2ndFlrSF'
print(fullListNumbers)

In [ ]:
fullListNumbers.remove('SalePrice')
linearRegression(datasetR,fullListNumbers,'SalePrice')

<b> Intégration des valeurs catégoricielles </b>

In [ ]:
allcatColumns = list(datasetR.select_dtypes(include=['object']).columns)
print(allcatColumns)

In [ ]:
test = datasetR.groupby(datasetR['Alley']).size().sort_values(ascending=False)
sns.barplot(x=list(test.index), y=list(test.iloc[0:]))
print('Alley' + ' - nombre de catégories : ' + str(len(test)) + ' - Pourcentage de la valeur 1 : ' + str(test[0]/len(datasetR)))

Nous trions mainteant parmis les données catégoricielles. Il faut d'abord retirer celles ne variant pas assez ou ayant trop de valeurs différentes.

In [ ]:
for i in allcatColumns:
    test = datasetR.groupby(datasetR[i]).size().sort_values(ascending=False)
    print(i + ' - nombre de catégories : ' + str(len(test)) + ' - Pourcentage de la valeur 1 : '+ str(test.iloc[0]/len(datasetR)))

Parce qu'une même valeur est rencontrée dans plus de 80% des cas, ou un nombre de catégories trop importantes est constatée nous supprimons ces élements : Alley,BldgType,BsmtCond,BsmtFinType2,CentralAir,Condition1,Condition2,Electrical,ExterCond,Exterior1st,Exterior2nd,Fence,
Functional,GarageCond,GarageQual,Heating,LandContour,LandSlope,MSSubClass,MiscFeature,MoSold,Neighborhood,PavedDrive,PoolQC,
RoofMatl,SaleCondition,SaleType,Street  

In [ ]:
toDelete = ['Alley','BldgType','BsmtCond','BsmtFinType2','CentralAir','Condition1','Condition2','Electrical','ExterCond','Exterior1st','Exterior2nd','Fence','Functional','GarageCond','GarageQual','Heating','LandContour','LandSlope','MSSubClass','MiscFeature','MoSold','Neighborhood','PavedDrive','PoolQC','RoofMatl','SaleCondition','SaleType','Street']  
for i in toDelete:
    allcatColumns.remove(i)

print(allcatColumns)

In [ ]:
def linearRegression(datasetR,fit,toPredict):
    fullListNumbers = list(datasetR.select_dtypes(include=['int64','float64']).columns)
    fullListNumbers = fullListNumbers[:-1]
    
    scaler = preprocessing.StandardScaler()
    dfStand = scaler.fit_transform(datasetR[fullListNumbers])
    dfStand = pd.DataFrame(dfStand, columns=fullListNumbers)
    datasetR[fullListNumbers] = dfStand
    
    allcatColumns = list(datasetR[fit].select_dtypes(include=['object']).columns)
    dummy_cols = pd.DataFrame()
    for col in allcatColumns:
        col_dummies = pd.get_dummies(datasetR[col],prefix=str(col))
        datasetR = pd.concat([datasetR, col_dummies], axis=1)
        del datasetR[col]
        fit = fit + list(col_dummies.columns)
        fit.remove(col)

    cut = round(len(datasetR)*0.8)
    dsTrain = datasetR.iloc[0:cut,:]
    dsTest = datasetR.iloc[cut:,:]
    
    lr = linear_model.LinearRegression() 
    lr.fit(dsTrain[fit],dsTrain[toPredict])
    prediction = lr.predict(dsTest[fit])
    rmse = mean_squared_error(prediction,dsTest[toPredict])**(1/2)
    return rmse

fullcolumnslist =  fullListNumbers + allcatColumns

In [ ]:
fullcolumnslist =  fullListNumbers + allcatColumns

result = linearRegression(datasetR,['OverallQual'],'SalePrice')

newcolumns = ['OverallQual']
for i in range(len(fullcolumnslist)):
    fit = fullcolumnslist[0:i+1]
    test = linearRegression(datasetR,fit,'SalePrice')
    if test < result:
        newcolumns.append(fullcolumnslist[i])
        result = test
        
print(newcolumns)

In [ ]:
linearRegression(datasetR,newcolumns,'SalePrice')

A vous ! Adaptez la fonction pour tester votre modèle sur le jeu de données d'entrainement du concours ! 

In [ ]:
datasetR[fullcolumnslist]